In [13]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import requests
from urllib.parse import quote

In [14]:
df = pd.read_csv('서울경찰서1.csv')
df.head()

,경찰서명,주소
0,경찰청 서울특별시경찰청 서울중부경찰서,서울특별시 중구 수표로 27
1,경찰청 서울특별시경찰청 서울종로경찰서,서울특별시 종로구 율곡로 46
2,경찰청 서울특별시경찰청 서울남대문경찰서,서울특별시 중구 한강대로 410
3,경찰청 서울특별시경찰청 서울서대문경찰서,서울특별시 서대문구 통일로 113
4,경찰청 서울특별시경찰청 서울혜화경찰서,서울특별시 종로구 창경궁로 112-16


In [15]:
with open('../04.지도시각화/kakaoapikey.txt') as file:
    kakao_key = file.read()

In [16]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [17]:
lng_list, lat_list = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    lng_list.append(float(result['documents'][0]['x']))
    lat_list.append(float(result['documents'][0]['y']))

In [20]:
df['위도'] = lat_list
df['경도'] = lng_list
df.head(5)

,경찰서명,주소,위도,경도
0,경찰청 서울특별시경찰청 서울중부경찰서,서울특별시 중구 수표로 27,37.563632,126.990021
1,경찰청 서울특별시경찰청 서울종로경찰서,서울특별시 종로구 율곡로 46,37.575876,126.984551
2,경찰청 서울특별시경찰청 서울남대문경찰서,서울특별시 중구 한강대로 410,37.554811,126.973549
3,경찰청 서울특별시경찰청 서울서대문경찰서,서울특별시 서대문구 통일로 113,37.565085,126.967074
4,경찰청 서울특별시경찰청 서울혜화경찰서,서울특별시 종로구 창경궁로 112-16,37.571768,126.999273


In [19]:
df.to_csv('서울경찰서위도경도1.csv', index=False)

In [21]:
import folium, json

In [22]:
filename = '../04.지도시각화/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(filename, encoding='utf-8'))

In [28]:
map = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=12)
for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        icon = folium.Icon(color='black', icon='flag'),
        popup=folium.Popup(df.주소[i], max_width=350),
        tooltip=df.경찰서명[i]
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">서울소재 경찰서 위치</h3>'    
map.get_root().html.add_child(folium.Element(title_html)) 
map